In [ ]:
!pip install PyDrive

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1YDTb0DduoP_kQwmEm3d5CEQk3wDyPhfe'})
download.GetContentFile('FER 2013.zip')



In [ ]:
ls

In [ ]:
import os
import zipfile

local_zip = '/content/FER 2013.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

base_dir="/content/FER 2013/"

img_size = 299
plt.figure(0, figsize=(12,20))
ctr = 0

for expression in os.listdir(base_dir+"train/"):
    for i in range(1,6):
        ctr += 1
        plt.subplot(7,5,ctr)
        img = load_img(base_dir+"train/" + expression + "/" +os.listdir(base_dir+"train/"+ expression)[i], target_size=(img_size, img_size))
        plt.imshow(img)

plt.tight_layout()
plt.show()

In [ ]:
for expression in os.listdir(base_dir+"train/"):
    print(str(len(os.listdir(base_dir+"train/" + expression))) + " " + expression + " images")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
#from livelossplot.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
img_size = 299
batch_size = 32




datagen_train = ImageDataGenerator(rescale = 1.0/255., 

     featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,   # randomly flip images
    zoom_range=0.2
  )

train_generator = datagen_train.flow_from_directory(base_dir+"train/",
                                                    target_size=(img_size,img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical'
                                                    
                                                    )

datagen_validation  = ImageDataGenerator( rescale = 1.0/255.
                                         
    
   
  )
validation_generator = datagen_validation.flow_from_directory(base_dir+"test/",
                                                    target_size=(img_size,img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False
                                                             )

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

  
from tensorflow.keras.applications.xception import Xception

img_width=299
img_height=299
pre_trained_model = Xception(input_shape=(img_width,img_height, 3), weights='imagenet', include_top=False)


for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()



#last_layer = pre_trained_model.get_layer('mixed7')
#print('last layer output shape: ', last_layer.output_shape)
#last_output = last_layer.output

In [ ]:
%%time
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 

from tensorflow.keras.regularizers import l2


x = layers.Flatten()(pre_trained_model.output)
x = layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001),
           bias_regularizer=l2(0.001))(x)
x=layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)                  

x = layers.Dense(512, activation='relu',kernel_regularizer=l2(0.001),
           bias_regularizer=l2(0.001))(x)
x=layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x) 

x = layers.Dense (7, activation='softmax')(x)            

model = Model( pre_trained_model.input, x)

input_shape=(299,299,3)
model.build(input_shape)

#model.build(input_shape) 
# Définition de l'optimizer (avec quelques paramètres qu'il faudra adapter à ses besoins)
#opt = RAdam(lr=0.1,total_steps=10000, warmup_proportion=0.1, min_lr=1e-6)
opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 30
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size




reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                             patience=2, min_lr=0.0000001, mode='auto')
callbacks = [reduce_lr]

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
   callbacks=callbacks
)




In [ ]:


#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
num_of_test_samples=7178
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
num_classes = len(emotion_labels)
Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(validation_generator.classes, y_pred)
print(cm)
cm_normalised = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.set(font_scale=1.5) 
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(cm_normalised, annot=True, linewidths=0, square=True, 
                    cmap="Greens", yticklabels=emotion_labels, xticklabels=emotion_labels, vmin=0, vmax=np.max(cm_normalised), 
                    fmt=".2f", annot_kws={"size": 20})
ax.set(xlabel='Predicted label', ylabel='True label')

fine-tuning


In [ ]:
# reset our data generators
train_generator.reset()
validation_generator.reset()
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in pre_trained_model.layers[115:]:
	layer.trainable = True
# loop over the layers in the model and show which ones are trainable
# or not
for layer in pre_trained_model.layers:
	print("{}: {}".format(layer, layer.trainable))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint


opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 9
# 20 epochs et meme 15 suffira
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size


checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1, save_best_only=True)



reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                             patience=2, min_lr=0.0000001, mode='auto')
callbacks = [reduce_lr,checkpoint]

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
   callbacks=callbacks
)

In [ ]:
ls

In [ ]:
from google.colab import files
files.download("model_weights.h5")

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
ls

In [ ]:
files.download("model.json")

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
num_of_test_samples=7178
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
num_classes = len(emotion_labels)
Y_pred = model.predict_generator(validation_generator, num_of_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(validation_generator.classes, y_pred)
print(cm)
cm_normalised = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.set(font_scale=1.5) 
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(cm_normalised, annot=True, linewidths=0, square=True, 
                    cmap="Greens", yticklabels=emotion_labels, xticklabels=emotion_labels, vmin=0, vmax=np.max(cm_normalised), 
                    fmt=".2f", annot_kws={"size": 20})
ax.set(xlabel='Predicted label', ylabel='True label')

In [ ]:
from sklearn.metrics import classification_report

test_steps=validation_generator.n//validation_generator.batch_size+1

print("[INFO] evaluating after fine-tuning network ...")
validation_generator.reset()
predIdxs = model.predict_generator(validation_generator,
	steps=test_steps)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(validation_generator.classes, predIdxs,
	target_names=validation_generator.class_indices.keys(), zero_division=1))


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(299, 299))
  yy = image.img_to_array(img)
  yy = np.expand_dims(yy, axis=0)

  images = np.vstack([yy])
  classes = model.predict(images, batch_size=10)
  print(fn)

  print(np.argmax(classes[0]))